In [1]:
import implicit
import pandas as pd
from scipy import sparse
import numpy
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.cross_validation import random_train_test_split
from sklearn.externals import joblib
import implicit

In [2]:
%time df = pd.read_csv('orders.csv', usecols = ['customerid','productid'], dtype = {'customerid':str, 'productid':str})

CPU times: user 1.14 s, sys: 781 ms, total: 1.92 s
Wall time: 1.89 s


In [3]:
uniqueCustomer = df['customerid'].unique()
uniqueProducts = df['productid'].unique()

print('Total number of unique customers : %.0f' % len(uniqueCustomer))
print('Total number of unique products  : %.0f' % len(uniqueProducts))

In [ ]:
customerDictionary = dict(enumerate(uniqueCustomer.flatten(), 1))
customerDictionaryInv = {v: k for k, v in customerDictionary.items()}
productDictionary = dict(enumerate(uniqueProducts.flatten(), 1))
productDictionaryInv = {v: k for k, v in productDictionary.items()}

rowIdx = [None] * len(df.index)
colIdx = [None] * len(df.index)
data = [None] * len(df.index)
for index, row in df.iterrows():
    custidx = customerDictionaryInv[row.customerid]
    productidx = productDictionaryInv[row.productid]
    
    rowIdx[index] = custidx
    colIdx[index] = productidx
    data[index] = 1
    
    if index % 50000 == 0:
        print("%s / %s" % (index, len(df.index)))
    
%time orders = sparse.csr_matrix((data, (rowIdx,colIdx)))

print(repr(orders))

0 / 2129758
50000 / 2129758
100000 / 2129758
150000 / 2129758
200000 / 2129758
250000 / 2129758
300000 / 2129758
350000 / 2129758
400000 / 2129758
450000 / 2129758
500000 / 2129758
550000 / 2129758
600000 / 2129758
650000 / 2129758
700000 / 2129758
750000 / 2129758
800000 / 2129758
850000 / 2129758
900000 / 2129758
950000 / 2129758
1000000 / 2129758
1050000 / 2129758
1100000 / 2129758
1150000 / 2129758
1200000 / 2129758
1250000 / 2129758
1300000 / 2129758
1350000 / 2129758
1400000 / 2129758
1450000 / 2129758
1500000 / 2129758
1550000 / 2129758
1600000 / 2129758
1650000 / 2129758
1700000 / 2129758
1750000 / 2129758
1800000 / 2129758
1850000 / 2129758
1900000 / 2129758
1950000 / 2129758
2000000 / 2129758
2050000 / 2129758
2100000 / 2129758
CPU times: user 891 ms, sys: 141 ms, total: 1.03 s
Wall time: 1.05 s
<721956x404838 sparse matrix of type '<class 'numpy.int64'>'
	with 2097754 stored elements in Compressed Sparse Row format>


In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=50)
%time model.fit(orders)

 13%|█▎        | 2.0/15 [02:05<13:08, 60.67s/it]